<h1 style="text-align:center">BIDIRECTIONAL</h1>

NAMA : GAYUH FIKRI KHUSNAELI

NIM : 21110024

# Import Dataset

In [ ]:
import pandas as pd
import string
import numpy as np

# Karena dataset bentuknya file txt, bukan csv jadi tidak bisa kita buka dengan pandas. Kita harus modifikasi filenya supaya bisa dibaca oleh pandas
# Buka file
file = open("/content/drive/MyDrive/data set/ind-eng/ind.txt").read()

# Buat data dictionary untuk diubah menjadi pandas datafrmae
data = {
    'ind':[],
    'eng':[]
}


for idx, kalimat in enumerate(file.split("\n")): # looping data setiap yang berakhiran '\n'
  kalimat = kalimat.split("\t")
  if len(kalimat) == 3:
    data['ind'].append(kalimat[1])
    data['eng'].append(kalimat[0])

# Membuat dataframe
df = pd.DataFrame(data)
df.head()

,ind,eng
0,Hai.,Hi.
1,Lari!,Run!
2,Lari!,Run.
3,Siapa?,Who?
4,Wow!,Wow!


In [ ]:
# Melihat 5 data terakhir
df.tail()

,ind,eng
13006,"Ketika dia sudah selesai menyetrika, Mary mema...","When she was finished ironing, Mary switched o..."
13007,"Ketika Tom berdiri untuk pidato, pikirannya ti...","When he got up to make his speech, Tom's mind ..."
13008,"Januari, Februari, Maret, April, Mei, Juni, Ju...","January, February, March, April, May, June, Ju..."
13009,"Irene Pepperberg, seorang peneliti di Universi...","Irene Pepperberg, a researcher at Northwestern..."
13010,Jika seseorang tidak berkesempatan untuk mengu...,If a person has not had a chance to acquire hi...


In [ ]:
# Jumlah baris dan kolom
df.shape

(13011, 2)

# Preprocessing Data

In [ ]:
# Data yang bernilai null
df.isna().sum()

ind    0
eng    0
dtype: int64

In [ ]:
# Kita hanya menghapus karakter '!@?.,' dan mengecilkan semua karakter
# Jika ingin menambahkan metode cleaning text yang lain tidak apa-apa
# Di bawah ini fungsi untuk menghapus punctuation dan mengecilkan huruf
def clean(text):
  no_punct = text.translate(str.maketrans('','', '!@?.,'))
  lower_text = no_punct.lower()
  return lower_text

In [ ]:
df_clean = df.copy()

# Menggunakan fungsi clean pada kolom ind dan eng
df_clean.ind = df_clean.ind.apply(clean)
df_clean.eng = df_clean.eng.apply(clean)
df_clean.head()

,ind,eng
0,hai,hi
1,lari,run
2,lari,run
3,siapa,who
4,wow,wow


In [ ]:
# Jumlah kalimat yang duplikat di masing-masing bahasa
print('Jumlah kalimat duplikat di ind : ', df_clean.duplicated(subset='ind').sum())
print('Jumlah kalimat duplikat di eng : ', df_clean.duplicated(subset='eng').sum())

Jumlah kalimat duplikat di ind :  1278
Jumlah kalimat duplikat di eng :  987


In [ ]:
# Untuk tidak menggunakan memori yang terlalu banyak dalam membuat model, kita akan hapus kalimat-kalimat yang duplikat
# Tetapi jika tidak dihapus juga tidak apa-apa, karena bisa jadi dalam bahasa indonesia kalimatnya sama tapi dalam bahasa inggris berbeda begitu pun sebaliknya

# Menghapus data yang memiliki duplikat pada kolom 'ind' dengan mempertahankan data yang lebih awal
df_clean.drop_duplicates(subset='ind', keep='first', inplace=True, ignore_index=True)

# Menghapus data yang memiliki duplikat pada kolom 'eng' dengan mempertahankan data yang lebih awal
df_clean.drop_duplicates(subset='eng', keep='first', inplace=True, ignore_index=True)

print('Jumlah kalimat duplikat di ind : ', df_clean.duplicated(subset='ind').sum())
print('Jumlah kalimat duplikat di eng : ', df_clean.duplicated(subset='eng').sum())
print('Jumlah data di dataset : ', df_clean.shape[0])

Jumlah kalimat duplikat di ind :  0
Jumlah kalimat duplikat di eng :  0
Jumlah data di dataset :  10964


In [ ]:
# Fungsi untuk mencari jumlah kata terbanyak di setiap kalimat
def max_lenght_sentences(data):
  length = [len(x.split()) for x in data]
  return max(length)

max_length_ind = max_lenght_sentences(df_clean.ind.values)
max_length_eng = max_lenght_sentences(df_clean.eng.values)
print('Kalimat terpanjang di ind memiliki kata sebanyak : ', max_length_ind)
print('Kalimat terpanjang di eng memiliki kata sebanyak : ', max_length_eng)

Kalimat terpanjang di ind memiliki kata sebanyak :  25
Kalimat terpanjang di eng memiliki kata sebanyak :  32


In [ ]:
# Karena keterbatasan memori yang bisa dipakai, kita hanya akan menggunakan data ind yang memiliki jumlah kata tidak lebih dari 10
# fungsi untuk mencari index data yang jumlah katanya tidak lebih dari 10 dan yang terdapat karakter angka
def filter_sentence(sentences, threshold):
  get_idx = list()
  for index, sentence in enumerate(sentences):
    if len(sentence.split()) > threshold:
      get_idx.append(index)
    if any(char.isdigit() for char in sentence) is True and index not in get_idx: # Untuk cek apakah kalimat ada karakter angka
      get_idx.append(index)

  return get_idx # akan mengembalikan index data yang jumlah katanya lebih dari 10 dan yang memeliki karakter angka

# Hapus berdasarkan index yang didapatkan
df_clean = df_clean.drop(filter_sentence(df_clean.ind.values, 10), axis=0)

max_length_ind = max_lenght_sentences(df_clean.ind.values)
max_length_eng = max_lenght_sentences(df_clean.eng.values)
print('Kalimat terpanjang di ind memiliki kata sebanyak : ', max_length_ind)
print('Kalimat terpanjang di eng memiliki kata sebanyak : ', max_length_eng)
print('Jumlah data di dataset : ', df_clean.shape[0])

Kalimat terpanjang di ind memiliki kata sebanyak :  10
Kalimat terpanjang di eng memiliki kata sebanyak :  17
Jumlah data di dataset :  10579


# Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer

# Fungsi untuk melakukan tokenizer
def create_tokenizer(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)
  return tokenizer

# Acak baris di dataset
df_clean = df_clean.sample(frac=1).reset_index(drop=True)

# Ubah data ind dan eng menjadi array satu dimensi
bhs_ind = df_clean.ind.values
bhs_eng = df_clean.eng.values

# Melakukan tokenizer
token_ind = create_tokenizer(bhs_ind)
token_eng = create_tokenizer(bhs_eng)

# Jumlah kosakata
vocab_ind_size = len(token_ind.word_index)
vocab_eng_size = len(token_eng.word_index)

print('Dataset ind mempunyai kosakata sebanyak : ', vocab_ind_size)
print('Dataset eng mempunyai kosakata sebanyak : ', vocab_eng_size)

Dataset ind mempunyai kosakata sebanyak :  5396
Dataset eng mempunyai kosakata sebanyak :  4630


# Vectorizer

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Fungsi untuk mengubah data menjadi vektor
def encode_sequences(tokenizer, data, max_lenght_sentences):
  vector_bhs = tokenizer.texts_to_sequences(data)
  vector_bhs = pad_sequences(vector_bhs, maxlen=max_lenght_sentences, padding='post')
  return vector_bhs

# Melakukan vectorizer
# parameter max_lenght kita gunakan yang paling besar yaitu 17 miliki eng
vector_ind = encode_sequences(token_ind, bhs_ind, max_length_ind)
vector_eng = encode_sequences(token_eng, bhs_eng, max_length_eng)

print("Sebelum dilakukan vectorizer :\n=>",bhs_ind[900])
print("Setelah dilakukan vectorizer :\n=>",vector_ind[900])

Sebelum dilakukan vectorizer :
=> tom pergi mengamati burung
Setelah dilakukan vectorizer :
=> [   1   24 2995  351    0    0    0    0    0    0]


# Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, Embedding, RepeatVector, Bidirectional, Dropout, GRU

model = Sequential()
model.add(Embedding(vocab_ind_size+1, 256, input_length = vector_ind.shape[1], input_shape=vector_ind.shape[1:]))
model.add(Bidirectional(GRU(256)))
model.add(RepeatVector(max_length_eng))
model.add(Bidirectional(GRU(256, return_sequences=True)))
model.add(TimeDistributed(Dense(1024,activation='linear')))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocab_eng_size+1,activation='softmax')))

In [ ]:
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

model.compile(loss= sparse_categorical_crossentropy, optimizer=Adam(learning_rate=0.001), metrics= ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 256)           1381632   
                                                                 
 bidirectional (Bidirection  (None, 512)               789504    
 al)                                                             
                                                                 
 repeat_vector (RepeatVecto  (None, 17, 512)           0         
 r)                                                              
                                                                 
 bidirectional_1 (Bidirecti  (None, 17, 512)           1182720   
 onal)                                                           
                                                                 
 time_distributed (TimeDist  (None, 17, 1024)          525312    
 ributed)                                               

In [ ]:
history = model.fit(vector_ind, vector_eng, batch_size=5, epochs=30, validation_split=0.2)

Epoch 1/30
1693/1693 [==============================] - 37s 15ms/step - loss: 2.1656 - accuracy: 0.7009 - val_loss: 1.9354 - val_accuracy: 0.7199
Epoch 2/30
1693/1693 [==============================] - 21s 12ms/step - loss: 1.8242 - accuracy: 0.7248 - val_loss: 1.7623 - val_accuracy: 0.7326
Epoch 3/30
1693/1693 [==============================] - 22s 13ms/step - loss: 1.5724 - accuracy: 0.7422 - val_loss: 1.6398 - val_accuracy: 0.7425
Epoch 4/30
1693/1693 [==============================] - 21s 12ms/step - loss: 1.3564 - accuracy: 0.7575 - val_loss: 1.5587 - val_accuracy: 0.7522
Epoch 5/30
1693/1693 [==============================] - 22s 13ms/step - loss: 1.1819 - accuracy: 0.7727 - val_loss: 1.5507 - val_accuracy: 0.7494
Epoch 6/30
1693/1693 [==============================] - 21s 13ms/step - loss: 1.0384 - accuracy: 0.7863 - val_loss: 1.5391 - val_accuracy: 0.7579
Epoch 7/30
1693/1693 [==============================] - 22s 13ms/step - loss: 0.9209 - accuracy: 0.7989 - val_loss: 1.5538 -

# Prediksi

In [ ]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = ''
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
kalimat_test = ['tom pergi untuk menemuimu tiga jam yang lalu']
vector_test = encode_sequences(token_ind, kalimat_test, max_length_ind)
y_pred = model.predict(vector_test)
print(logits_to_text(y_pred[0], token_eng))

1/1 [==============================] - 0s 27ms/step
tom left to you you three hours accident         


In [ ]:
df_clean.iloc[8000,1]

'tom left to meet you three hours ago'

# Evaluasi

Untuk melakukan evaluasi dalam contoh kasus translasi text ini bisa menggunakan BLUE score. Teman-teman bisa mempelajarinya di refrensi link ini https://machinelearningmastery.com/calculate-bleu-score-for-text-python/